# Jison

## Instalação

A instalação pode ser feita utilizando node (npm) ao executar o comando:

```
npm install jison -g
```

## Especificando a linguagem para uma calculadora

O processo de _parsing_ de uma linguagem comumente envolve duas fases: análise léxica (criação de tokens) e parsing, pelas quais as ferramentas Lex/Yacc e Flex/Bison são tão famosas. 

Jison permite especifiar um parser utilizando arquivos separados para regras léxicas e para a gramática ou um só arquivo, com as regras léxicas juntas com a gramátita.

### Parser para uma calculadora

O código a seguir, para o arquivo `calculadora.jison` apresenta uma linguagem para uma calculadora.

```
/* Linguagem para avaliação de expressões matemáticas */

/* regras léxicas */
%lex

%%
\s+                   /* pula espaços em branco */
[0-9]+("."[0-9]+)?\b  return 'NUMBER';
"*"                   return '*';
"/"                   return '/';
"-"                   return '-';
"+"                   return '+';
"^"                   return '^';
"("                   return '(';
")"                   return ')';
"PI"                  return 'PI';
"E"                   return 'E';
<<EOF>>               return 'EOF';

/lex

/* associações e precedência de operadores */

%left '+' '-'
%left '*' '/'
%left '^'
%left UMINUS

%start expressions

%% /* gramática */

expressions
    : e EOF
        {print($1); return $1;}
    ;

e
    : e '+' e
        {$$ = $1+$3;}
    | e '-' e
        {$$ = $1-$3;}
    | e '*' e
        {$$ = $1*$3;}
    | e '/' e
        {$$ = $1/$3;}
    | e '^' e
        {$$ = Math.pow($1, $3);}
    | '-' e %prec UMINUS
        {$$ = -$2;}
    | '(' e ')'
        {$$ = $2;}
    | NUMBER
        {$$ = Number(yytext);}
    | E
        {$$ = Math.E;}
    | PI
        {$$ = Math.PI;}
    ;
```

Crie o parser utilizando o comando:

```
jison calculadora.jison
```

O comando cria o arquivo `calculadora.js` no diretório atual. O script pode, então, ser usado para fazer parsing de um arquivo. Exemplo:

```
# arquivo teste.calc
2 ^ 32 / 1024 
```

Para 'compilar e executar' o programa `teste.calc`:

```
node calculadora teste.calc
```

A saída é:

```
4194304
```

ou seja, o resultado de $\frac{2^{32}}{1024}$.

Também é possível utilizar a API do parser. Conteúdo do arquivo `modulo.js`:

```javascript
var parser = require("./calculadora").parser;

function exec (input) {
    return parser.parse(input);
}

var vinte = exec("4 * 5");
```

E, para executar, o comando:

```
node modulo
```

## Especificando uma linguagem

O arquivo `.jison` possui uma estrutura básica:

```
/* regras léxicas */
%lex
...
%%
/lex

/* regras sintáticas (gramática) */
%%
...
```

A composição do arquivo é:
* regras léxicas; e
* regras gramaticais



## Regras léxicas

As regras léxicas são compostas por duas partes:
1. conjunto de caracteres ou expressão regular que define a estrutura do token (ou seja, um padrão)
2. nome do token

**Exemplo:**

```
[0-9]+("."[0-9]+)?\b  return 'NUMBER';
```

A primeira parte `[0-9]+("."[0-9]+)?\b` é uma expressão regular que combina com tokens que tenham essa estrutura: números de 0 a 9, repetidos uma ou mais vezes, seguidos, opcionalmente, de "." e números de 0 a 9 repetidos uma ou mais vezes. O metacaractere `\b` indica que o número deve estar ao final de uma palavra. 

A segunda parte indica que o identificador do token em questão é `NUMBER` (obrigatório o uso de `return` e aspas).

**Exemplo:**

```
"+"                   return '+';
```

O token `+` é representado pelo próprio símbolo.

### Precedência de operadores

As regras léxicas também incluem a definição de precedência de operadores:

```
%left '+' '-'
%left '*' '/'
%left '^'
%left UMINUS
```

A declaração `%left` torna os operadores com associação à esquerda e a declaração `%right` os torna com associação à direita. A declaração `%nonassoc` indica que é um erro encontrar o mesmo operador duas vezes na sequência.

Considere a expressão `1 - 2 + 3`. Se for usada associação à esquerda, a expressão é avaliada como `(1 - 2) + 3`. Se for usada associação à esquerda, a expressão é avaliada como `1 - (2 + 3)`. Para operadores, geralmente será usada associação à esquerda. Para atribuição, geralmente é usada associação à direita.

A ordem dessas declarações influencia o grau de precedência. No exemplo, todos têm associação à esquerda, mas os operadores `+` e `-` têm menor precedência, enquanto o operador `UMINUS` tem maior precedência.

Assim, as regras indicam que os operadores `+` e `-` têm a mesma precedência, com associação à esquerda, a qual é inferior à precedência de `*` e `/`.

## Regras sintáticas (gramaticais)

A sintaxe de uma regra gramatical é:

```
resultado:  componentes...
            ;
```

Onde:
* `resultado` é o símbolo não-terminal que é descrito na regra; e
* `componentes` é composto por símbolos terminais e não-terminais

**Exemplo:**

```
expressao:  expressao '+' expressao
          | expressao '-' expressao
          | expressao '*' expressao
          | expressao '/' expressao
          ;
```

A regra indica que o não-terminal `expressao` é formado por múltiplas regras, separadas por `|`, o que indica disjunção.

### Regras recursivas

Uma regra pode ser recursiva à esquerda ou à direita. Exemplo:

```
expressao:   numero
           | expressao '+' numero
           ;
```

A regra que define o não-terminal `expressao` é recursiva à esquerda, pois este não-terminal aparece novamente no lado direito, mais à esquerda.

Outro exemplo:

```
expressao:   numero
           | numero '+' expressao
           ;
```

A regra agora é recursiva à direita, pois o não-terminal aparece no lado direito, mais à direita.

Em Jison, utilize regras recursivas à esquerda.

## Semântica

A semântica de uma linguagem é determinada pelos valores associados com vários tokens ou grupos de tokens e pelas ações executadas quando vários agrupamentos são reconhecidos. Exemplo:

```
e
    : e '+' e
        {$$ = $1+$3;}
    | e '-' e
        {$$ = $1-$3;}
```

O símbolo não-terminal `e` é composto por duas regras:
* `e '+' e`
* `e '-' e`

Na sequência de cada regra está presente uma ação, entre `{}`. Para a regra da soma, a ação é `$$ = $1 + $3` e é interpretada da seguinte forma:
* a regra semântica tem três grupos de tokens: `e`, `+` e `e`, suas posições sendo: 1, 2 e 3 (índice iniciando-se em 1)
* `$1` refere-se ao primeiro `e`
* `$3` refere-se ao último `e`
* `$n` refere-se ao n-ésimo grupo de tokens da regra
* `$$` refere-se ao **valor semântico** da regra

A ação padrão é `$$ = $1`, ou seja, o valor semântico de uma regra é o valor do primeiro grupo dos tokens da regra.

Jison considera que o valor de `$n` é uma `string`, por isso, conversões de dados podem ser necessárias, utilizando-se funções do JavaScript. É o que ocorre com a regra:

```
e: NUMBER {$$ = Number(yytext);}
```

Neste caso, a função `Number()` é usada para converter `yytext` em um valor numérico. `yytext` representa o texto que representa o grupo de tokens. Como a regra aplica-se a números, então `yytext` contém o número, em si.

### A declaração `%start`

A declaração `%start` indica o símbolo inicial da gramática. Exemplo:

```
%start expressions
```

Indica que o símbolo inicial da gramática é o não-terminal `expressions`.